# Clasificar proyectos CORFO en salud u otro

El objetivo de este notebook es clasificar cada proyecto CORFO en `salud` si es que aborda algún tema (cualquiera) relacionado a salud, u `otro`, en caso contrario.

In [ ]:
!pip install openai
!pip install langchain
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import numpy as np
import pandas as pd
from csv import writer
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from google.colab import userdata

## Cargar datos

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/CORFO/Datos"
save_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/CORFO/Resultados"
df = pd.read_excel(os.path.join(data_dir, "DataInnovacion - InnovaChile Corfo.xlsx"),  skiprows=1)
df

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Inicio Actividad Económica,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final
0,23SN-242063,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77003755-7,Bebidas Saludables SpA,Mejorar las capacidades de las bebidas Masala ...,...,2019-05-20 00:00:00,Turismo,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,Sí,NaN,NaN,Sin tendencia
1,23SN-242058,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,76228136-8,Tecagromed SpA,TECFILMS: Desarrollo de un biomaterial en form...,...,2012-11-20 00:00:00,Comercio y retail,Intercambiar,Economía Circular,R2 Reducir,Uso y fabricación de productos más inteligentes,No,NaN,NaN,Química Verde (Green Chemestry)
2,23SN-242042,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77618332-6,Green Focus Solutions SpA,Kit molecular para la identificación rápida de...,...,2022-12-08 00:00:00,Biotecnológico,NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Nanotecnología
3,23SN-242041,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77704466-4,Zeroglut SpA,Evaluación de tecnologías de envasado para inc...,...,2023-02-21 00:00:00,Alimentos (excepto vitivinícola),Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia
4,23SN-242036,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77147033-5,La mar spa,Desarrollo de un pastel de jaibas listo para c...,...,2020-04-23 00:00:00,Comercio y retail,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9439,09ANT-7178,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,99576630-2,OXZO SOCIEDAD ANONIMA,introducción al mercado nacional de plataforma...,...,2004-12-01 00:00:00,Manufactura de no metálicos,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9440,09ANT-7131,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,77183150-8,METALCOP INGENIERIA LIMITADA,nuevo sistema apertura tapa balde para palas d...,...,1998-07-10 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9441,09ACS-7457,Desarrolla innovación,Subsidio,Apoyo Elaboración Capital Semilla,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,96691330-4,DICTUC S.A.,apoyo a la elaboración de estudios de preinver...,...,1994-03-02 00:00:00,Servicios de ingeniería o de conocimiento,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9442,09AAP-6741,Renuévate,Subsidio,Apoyo A La Atracción De Profesionales Y Técnicos,Capital Humano,FINALIZADO,Persona Jurídica constituida en Chile,76832220-1,ORION FORESTAL SA,desarrollo de sistemas de recolección de bioma...,...,2007-05-11 00:00:00,Forestal,Intercambiar,Economía Circular,NaN,Uso y fabricación de productos más inteligentes,No,NaN,Biocombustible,Sin tendencia


## Preprocesar datos

In [ ]:
df[df["Objetivo"].isna()]

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Inicio Actividad Económica,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final
2876,18ITE2-102868,Desarrolla innovación,Subsidio,Línea 2: Validación Y Empaquetamiento De Innov...,Validación y empaquetamiento de innovaciones,FINALIZADO,Persona Jurídica constituida en Chile,76276688-4,SOCIEDAD VÖLKER & AQUEA LIMITADA,estandarización de la producción de australcel...,...,2013-07-04 00:00:00,Agrícola (excepto vitivinícola),NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Sin tendencia
2916,18EUREID-105748,Desarrolla innovación con I+D,Subsidio,LÍNEA 1: I+D APLICADA EN EMPRESAS-CORFO EUREKA,I+D Aplicada en Empresas,FINALIZADO,Persona Jurídica constituida en Chile,76063790-4,INGENIERIA REDCO LIMITADA,sistema predictivo multi-fabricante de mantenc...,...,2004-03-01 00:00:00,Tecnologías de la información,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
5103,16PIUK-57404,Desarrolla innovación,Subsidio,Prototipos De Innovacion Corfo - Innova Uk,Prototipos de innovación,FINALIZADO,Persona Jurídica constituida en Chile,77762940-9,ANGLO AMERICAN SUR S.A.,manejo del agua y la energia para el desarroll...,...,2002-06-07 00:00:00,Minería y metalurgia extractiva,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia
5539,16CET2-63914,Entorno para la innovación,Subsidio,Centros De Extensionismo Tecnológico,Fortalece Pyme,FINALIZADO,Persona Jurídica constituida en Chile,70885500-6,UNIVERSIDAD DE TALCA,cetlog: centro de extensionismo logístico,...,1993-01-01 00:00:00,Educación,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
7389,12SSAF-17789,Desarrolla innovación,Subsidio,Subsidio Semilla De Asignación Flexible Para E...,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,81668700-4,UNIVERSIDAD TÉCNICA FEDERICO SANTA MARÍA,boosterup: programa de aceleración estrategica...,...,1993-01-01 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
7391,12SSAF-14855,Desarrolla innovación,Subsidio,Subsidio Semilla De Asignación Flexible Para E...,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,76109353-3,UDD VENTURES S.A.,fondo ssaf udd ventures,...,2010-08-20 00:00:00,Servicios de ingeniería o de conocimiento,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
7553,12PAE-16027,Entorno para la innovación,Subsidio,Apoyo Al Entorno Emprendedor,Apoyo al entorno del emprendimiento,FINALIZADO,Persona Jurídica constituida en Chile,96863740-1,FUNDES CHILE SPA,programa de emprendimiento femenino mundo mujer,...,1998-06-25 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
8066,11SUP-9599,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,babelverse,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
8229,11SUP-11576,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,patagonia sostenible,...,NaN,NaN,NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Sin tendencia
8299,11SSAF-9928,Desarrolla innovación,Subsidio,Subsidio Semilla De Asignación Flexible Para E...,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,70300000-2,FUNDACIÓN CHILE,fondo ssaf fundación chile,...,1993-01-01 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia


In [ ]:
df["Objetivo"].value_counts()

Información no pública                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1153
.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         108
sin informac

22 filas sin objetivo.

In [ ]:
df[df["Título"].isna()]

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Inicio Actividad Económica,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final
9014,10NTEC-7373,Entorno para la innovación,Subsidio,Línea del Apoyo al Extensionismo,Nodos,FINALIZADO,Persona Jurídica constituida en Chile,65386530-9,ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO ES...,NaN,...,2004-07-06 00:00:00,Asociaciones y organizaciones no empresariales...,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9266,10ANT-8103,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,96825800-1,COMERCIAL Y DISTRIBUIDORA BERTONATI S.A.,NaN,...,1997-07-23 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia


2 filas sin título.

In [ ]:
def merge_title_and_objectives(title, objective):
  if type(title) != str and type(objective) != str:
    return None
  if type(objective) != str or objective.lower().strip() in ["información no pública", ".", "sin información"]:
    return f"TÍTULO: {title}"
  return f"TÍTULO: {title}. OBJETIVO: {objective}"

df["glosa"] = df.apply(lambda row: merge_title_and_objectives(row["Título"], row["Objetivo"]), axis=1)

In [ ]:
df["glosa"]

0       TÍTULO: Mejorar las capacidades de las bebidas...
1       TÍTULO: TECFILMS: Desarrollo de un biomaterial...
2       TÍTULO: Kit molecular para la identificación r...
3       TÍTULO: Evaluación de tecnologías de envasado ...
4       TÍTULO: Desarrollo de un pastel de jaibas list...
                              ...                        
9439    TÍTULO: introducción al mercado nacional de pl...
9440    TÍTULO: nuevo sistema apertura tapa balde para...
9441    TÍTULO: apoyo a la elaboración de estudios de ...
9442    TÍTULO: desarrollo de sistemas de recolección ...
9443    TÍTULO: biomasa forestal del manejo sustentabl...
Name: glosa, Length: 9444, dtype: object

In [ ]:
df_valid = df[~df["glosa"].isna()]
len(df_valid)

9442

Se sacaron dos filas donde no había ni título ni objetivo.

In [ ]:
df_valid = df_valid[df_valid["Año Adjudicación"] >= 2012]
df_valid

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final,glosa
0,23SN-242063,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77003755-7,Bebidas Saludables SpA,Mejorar las capacidades de las bebidas Masala ...,...,Turismo,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,Sí,NaN,NaN,Sin tendencia,TÍTULO: Mejorar las capacidades de las bebidas...
1,23SN-242058,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,76228136-8,Tecagromed SpA,TECFILMS: Desarrollo de un biomaterial en form...,...,Comercio y retail,Intercambiar,Economía Circular,R2 Reducir,Uso y fabricación de productos más inteligentes,No,NaN,NaN,Química Verde (Green Chemestry),TÍTULO: TECFILMS: Desarrollo de un biomaterial...
2,23SN-242042,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77618332-6,Green Focus Solutions SpA,Kit molecular para la identificación rápida de...,...,Biotecnológico,NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Nanotecnología,TÍTULO: Kit molecular para la identificación r...
3,23SN-242041,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77704466-4,Zeroglut SpA,Evaluación de tecnologías de envasado para inc...,...,Alimentos (excepto vitivinícola),Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia,TÍTULO: Evaluación de tecnologías de envasado ...
4,23SN-242036,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77147033-5,La mar spa,Desarrollo de un pastel de jaibas listo para c...,...,Comercio y retail,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia,TÍTULO: Desarrollo de un pastel de jaibas list...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8000,11IEI-12756,Desarrolla innovación,Subsidio,Innovación Empresarial Individual,Innovación Empresarial Individual,FINALIZADO,Persona Jurídica constituida en Chile,76059703-1,SOCIEDAD DE INVERSIONES SIMA LIMITADA,desarrollo de tintas de impresión inkjet de gr...,...,Banca y sector financiero,Intercambiar,Economía Circular,R0 Rechazar,Uso y fabricación de productos más inteligentes,No,NaN,NaN,Sin tendencia,TÍTULO: desarrollo de tintas de impresión inkj...
8001,11IEI-12162,Desarrolla innovación,Subsidio,Innovación Empresarial Individual,Innovación Empresarial Individual,FINALIZADO,Persona Jurídica constituida en Chile,96930530-5,VINOTEC CHILE S A,desarrollo de productos de alto valor agregado...,...,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: desarrollo de productos de alto valor ...
8002,11IEI-11916,Desarrolla innovación,Subsidio,Innovación Empresarial Individual,Innovación Empresarial Individual,FINALIZADO,Persona Jurídica constituida en Chile,96596370-7,AGUAMARINA SPA.,desarrollo de solución tecnológica scab (siste...,...,Servicios empresariales administrativos y de a...,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: desarrollo de solución tecnológica sca...
8003,11FBCT-12991,Desarrolla innovación con I+D,Subsidio,Financiamiento Basal Transitorio Para Consorci...,Consorcios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,76571080-4,INNOVACION Y TECNOLOGIA VITIVINICOLA S.A,continuidad operativa del consorcio tecnológic...,...,Comercio y retail,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: continuidad operativa del consorcio te...


Sacamos además los proyectos previos al 2012 (usaremos del 2013 en adelante, pero igual dejamos los del 2012 por si acaso)

## Clasificación

Inicializar modelo

In [ ]:
api_key = userdata.get('openai_api_key')
chat = ChatOpenAI(model_name="gpt-4",
                    openai_api_key=api_key,
                    temperature=0,
                    max_tokens=3500)
chat

Le daremos al LLM una lista de instrucciones, pidiéndole clasificar cada proyecto en `cáncer`, `diabetes`, `cardiaca` u `otro`. Luego, le pasaremos una lista de proyectos para que clasifique. Iteraremos en bloques de 5 proyectos, e iremos guardando de inmediato los resultados.

In [ ]:
system_prompt = ("Eres un experto en evaluación de proyectos de investigación. " +
                  "Recibiras una lista de proyectos, cada uno empezando con un guión (-).  " +
                  "Algunos proyectos tienen un título y un objetivo, mientras otros sólo tienen título. " +
                  "Tu trabajo es determinar, para cada proyecto, si habla de salud u otro tema. " +
                  "Si el tema del proyecto es salud humana, debes responder 'salud'. " +
                  "Si el tema del proyecto no es salud humana, debes responder 'otro'. " +
                  "No pongas guiones al principio de cada respuesta. Sólo debes responder la clasificación correcta en cada línea.")

system_message_prompt = SystemMessage(content=system_prompt)
n_projects_prompt = 5

filepath = os.path.join(save_dir, "clasificaciones_corfo_salud_otro.csv")
if not os.path.exists(filepath):
  with open(filepath, "w") as fp:
    print("Creando archivo...")
    csv_writer = writer(fp)
    header = ["código", "glosa", "clasificación"]
    csv_writer.writerow(header)


for iii in tqdm(range(0, 3380, n_projects_prompt)):
  current_df = df_valid.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["glosa"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["Código Proyecto"].values, current_df["glosa"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

  0%|          | 0/676 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
100%|██████████| 676/676 [25:41<00:00,  2.28s/it]


## Revisar clasificaciones

In [ ]:
filepath = os.path.join(data_dir, "clasificaciones_corfo_salud.csv")
df_class = pd.read_csv(filepath)
df_class

,código,glosa,clasificación
0,18IS6-98611,TÍTULO: prototipo reciclaje inclusivo y colabo...,otro
1,18IS6-98607,TÍTULO: sistema de reciclaje de frutas para co...,otro
2,18IS6-98575,TÍTULO: escuela de cocina territorial y susten...,otro
3,18IS6-98557,"TÍTULO: ""el partido de mi barrio rancagua"", la...",otro
4,18IS6-98389,TÍTULO: modelo de mediación ambiental y certif...,otro
...,...,...,...
8000,18IS9-97652,TÍTULO: rcicla2: programa de formación ambient...,otro
8001,18IS6-98635,TÍTULO: senderismo y escalada para la superaci...,otro
8002,18IS6-98632,TÍTULO: agroinm integremos a inmigrantes en el...,otro
8003,18IS6-98630,TÍTULO: modelo de gestión participativa para l...,otro


In [ ]:
ids_1 = set(df_valid["Código Proyecto"].unique())
ids_2 = set(df_class["código"].unique())

ids_1 == ids_2

False

Todos los proyectos tienen clasificación :)

In [ ]:
df_class["clasificación"].value_counts()

otro                8436
otro.                787
- otro                74
cáncer                72
diabetes              33
cardiaca              31
- cáncer               5
diabetes.              1
- diabetes             1
diabetes, cáncer       1
cáncer.                1
Name: clasificación, dtype: int64

In [ ]:
df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()

<ipython-input-47-39a0d2e375fc>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()


In [ ]:
df_class["clasificación_normalizada"].value_counts()

otro                9297
cáncer                78
diabetes              35
cardiaca              31
diabetes, cáncer       1
Name: clasificación_normalizada, dtype: int64

In [ ]:
df_class.to_csv(os.path.join(save_dir, "clasificaciones_corfo_salud_otro_limpia.csv"), index=False)